colab 에서 gutenberg 패키지를 사용하기 위해서는, libdb5.3-dev 선행 설치가 필요하다.

In [0]:
!sudo apt install libdb5.3-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  db5.3-doc
The following NEW packages will be installed:
  libdb5.3-dev
0 upgraded, 1 newly installed, 0 to remove and 10 not upgraded.
Need to get 762 kB of archives.
After this operation, 3,146 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libdb5.3-dev amd64 5.3.28-13.1ubuntu1 [762 kB]
Fetched 762 kB in 0s (7,484 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libdb5.3-dev.
(Rea

In [0]:
!pip3 install --upgrade setuptools
!pip3 install gutenberg

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (40.8.0)
    100% |████████████████████████████████| 245kB 17.1MB/s 
    100% |████████████████████████████████| 348kB 26.0MB/s 
    100% |████████████████████████████████| 1.0MB 21.2MB/s 
    100% |████████████████████████████████| 51kB 22.8MB/s 
    100% |████████████████████████████████| 573kB 27.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/8e/cd/75/4bc6f16541a1b7a69b02168da567695b2271c23ac4a0a0a453
  Stored in directory: /root/.cache/pip/wheels/11/b8/b3/fa84db10bf8c563e4ba1a72837a0946d123f12adb34b164bf5
  Stored in directory: /root/.cache/pip/wheels/f9/71/46/604b8a4f0a04b513f5799c974b556c1de19a70fde41d25672b
  Stored in directory: /root/.cache/pip/wheels/15/35/25/dbcb848832ccb1a4b4ad23f529badfd3bce9bf88017f7ca510
Successfully built gutenberg bsddb3 alembic Mako


In [0]:
!pwd

/content


시간 절약을 위해, gutenberg db file 들을 미리 다운로드 받아놓고, google drive 를 mount 하여 연결한다.

* gutenberg db file 다운로드: https://drive.google.com/drive/folders/1Y6nMqJ-srDfflTuoniVdEzdWfkrShe4T

*   gutenberg db file  기본 경로 : ~/gutenberg_data/metadata/metadata.db

colab 상에서는, /content/gutenberg_data/metadata/metadata.db 에 위치하면 됨

In [0]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# unmount 를 원할 시 실행

#!fusermount -u gdrive
#!rmdir gdrive

fusermount: failed to unmount /content/gutenberg_data: Invalid argument


/content/gdrive 에 google drive 를 mount 하고,

 구글 드라이브 상의 /gutenberg/gutenberg_data 를 
 /content/gutenberg_data 위치로 symbolic link 연결한다


In [0]:
!ln -s /content/gdrive/My\ Drive/gutenberg/gutenberg_data /content/gutenberg_data

In [0]:
!ls -al /content/gutenberg_data/metadata/metadata.db

ls: cannot access '/content/gutenberg_data/metadata/metadata.db': No such file or directory


In [0]:
try:
    GUTENBERG = True
    from gutenberg.acquire import load_etext
    from gutenberg.query import get_etexts, get_metadata
    from gutenberg.acquire import get_metadata_cache
    from gutenberg.acquire.text import UnknownDownloadUriException
    from gutenberg.cleanup import strip_headers
    from gutenberg._domain_model.exceptions import CacheAlreadyExistsException
except ImportError:
    GUTENBERG = False
    print('Gutenberg is not installed. See instructions at https://pypi.python.org/pypi/Gutenberg')
#import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import TimeDistributed
import tensorflow.keras.callbacks
import tensorflow.keras.backend as K
import scipy.misc
import json

import os, sys
import re
import PIL
from PIL import ImageDraw

from tensorflow.keras.optimizers import RMSprop
import random
import numpy as np
import tensorflow as tf
from keras.utils import get_file

from IPython.display import clear_output, Image, display, HTML
try:
    from io import BytesIO
except ImportError:
    from StringIO import StringIO as BytesIO

미리 db 데이터를 넣어 놓아도,  cache.populate() 시 메타 데이타 땡겨오는데 시간이 꽤 걸린다.

꼼수가 있는데, 아래 코드블럭을 run 하고 바로 stop 하면 메타데이타(get_metadata) 검색은 잘 안되지만,


실제 텍스트 데이터 로드(load_etext) 는 잘 실행된다.

In [0]:
if GUTENBERG:
    cache = get_metadata_cache()
    try:
        cache.populate()
    except CacheAlreadyExistsException as e:
        print(e)
        pass

location: /root/gutenberg_data/metadata/metadata.db


서적 ID를 기반으로 실제 서적 데이터를 땡겨온다.


100 : 세익스피어 전집(희곡)

2981 : 카사노바 회고집(에세이)

[9296, 9798, 9881, 10462, 10799, 11364, 11889, 12180, 12398] : clarissa (여러 편의 편지 형태)


학습 텍스트는 원본에서 목차 등 불필요한 텍스트를 발라내게 된다. (text.split 부분)

In [0]:
shakespeare_id = 100
casanova_id = 2981 
clarissa_ids = [9296, 9798, 9881, 10462, 10799, 11364, 11889, 12180, 12398]

def load_etext_from(ids, filter_func):
  etext = '\n'.join([filter_func(strip_headers(load_etext(id))) \
                     for id in ids])
  return etext

In [0]:
shakespeare = load_etext_from([shakespeare_id], lambda text: text.split('\nTHE END', 1)[-1])
print(len(shakespeare))
shakespeare[:1000]

5528070


'\n\n\n\nALL’S WELL THAT ENDS WELL\n\n\nby William Shakespeare\n\n\n\nContents\n\nACT I\nScene I. Rossillon. A room in the Countess’s palace.\nScene II. Paris. A room in the King’s palace.\nScene III. Rossillon. A Room in the Palace.\n\n\nACT II\nScene I. Paris. A room in the King’s palace.\nScene II. Rossillon. A room in the Countess’s palace.\nScene III. Paris. The King’s palace.\nScene IV. Paris. The King’s palace.\nScene V. Another room in the same.\n\n\nACT III\nScene I. Florence. A room in the Duke’s palace.\nScene II. Rossillon. A room in the Countess’s palace.\nScene III. Florence. Before the Duke’s palace.\nScene IV. Rossillon. A room in the Countess’s palace.\nScene V. Without the walls of Florence.\nScene VI. Camp before Florence.\nScene VII. Florence. A room in the Widow’s house.\n\n\nACT IV\nScene I. Without the Florentine camp.\nScene II. Florence. A room in the Widow’s house.\nScene III. The Florentine camp.\nScene IV. Florence. A room in the Widow’s house.\nScene V. Ros

In [0]:
casanova = load_etext_from([casanova_id], lambda text: text.split('\nCASANOVA AT DUX', 1)[-1]) # from main contents
print(len(casanova))
casanova[:1000]

6685264


'\n\n An Unpublished Chapter of History, By Arthur Symons\n\n I\n The Memoirs of Casanova, though they have enjoyed the popularity of a bad reputation, have never had justice done to them by serious students of literature, of life, and of history. One English writer, indeed, Mr. Havelock Ellis, has realised that ‘there are few more delightful books in the world,’ and he has analysed them in an essay on Casanova, published in Affirmations, with extreme care and remarkable subtlety. But this essay stands alone, at all events in English, as an attempt to take Casanova seriously, to show him in his relation to his time, and in his relation to human problems. And yet these Memoirs are perhaps the most valuable document which we possess on the society of the eighteenth century; they are the history of a unique life, a unique personality, one of the greatest of autobiographies; as a record of adventures, they are more entertaining than Gil Blas, or Monte Cristo, or any of the imaginary travel

In [0]:
clarissa = load_etext_from(clarissa_ids, lambda text: text.split('\nTHE HISTORY OF CLARISSA HARLOWE', 1)[-1]) # from main contents
print(len(clarissa))
clarissa[:1000]

5173348


"\n\n\n\n\nLETTER I\n\nMISS ANNA HOWE, TO MISS CLARISSA HARLOWE JAN 10.\n\n\nI am extremely concerned, my dearest friend, for the disturbances that\nhave happened in your family. I know how it must hurt you to become\nthe subject of the public talk: and yet, upon an occasion so generally\nknown, it is impossible but that whatever relates to a young lady, whose\ndistinguished merits have made her the public care, should engage every\nbody's attention. I long to have the particulars from yourself; and of\nthe usage I am told you receive upon an accident you could not help; and\nin which, as far as I can learn, the sufferer was the aggressor.\n\nMr. Diggs, the surgeon, whom I sent for at the first hearing of the\nrencounter, to inquire, for your sake, how your brother was, told me,\nthat there was no danger from the wound, if there were none from the\nfever; which it seems has been increased by the perturbation of his\nspirits.\n\nMr. Wyerley drank tea with us yesterday; and though he is 

모델 저장에 쓸 파일명, char 임베딩 목록, chunk_size 등 메타데이타를 dict 로 묶어둔다.

In [0]:
# TODO : global char_to_index 추가 
def get_chars_index(etext):
  chars = list(sorted(set(etext)))
  char_to_idx = {ch: idx for idx, ch in enumerate(chars)}
  return chars, char_to_idx

def generate_meta_from(etext, model_name, chunk_size=160):
  etext_meta = {}
  etext_meta['model_name'] = model_name
  etext_meta['char'], etext_meta['char_to_idx'] = get_chars_index(etext)
  etext_meta['chunk_size'] = chunk_size
  return etext_meta
  
shakespeare_meta = generate_meta_from(shakespeare, 'shakespeare')
casanova_meta = generate_meta_from(casanova, 'casanova')
clarissa_meta = generate_meta_from(clarissa, 'clarissa')

In [0]:
def char_rnn_model(chunk_size, num_chars, num_layers, num_nodes=512, dropout=0.1):
    input = Input(shape=(chunk_size, num_chars), name='input')
    prev = input
    for i in range(num_layers):
        lstm = LSTM(num_nodes, return_sequences=True, name='lstm_layer_%d' % (i + 1))(prev)
        if dropout:
            prev = Dropout(dropout)(lstm)
        else:
            prev = lstm
    dense = TimeDistributed(Dense(num_chars, name='dense', activation='softmax'))(prev)
    model = Model(inputs=[input], outputs=[dense])
    optimizer = tf.train.RMSPropOptimizer(learning_rate=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


기 학습되어 google drive 에 저장된 모델을 로드하여 inference 해본다.



In [0]:
# prediction from saved weights
!ls -al /content/gutenberg_data/models/
basepath = '/content/gutenberg_data/models/02/'

def load_model(basepath, model_meta):
  model = char_rnn_model(None, len(model_meta['char']), num_layers=2, num_nodes=640, dropout=0) 
  model.load_weights(basepath + "/" + model_meta['model_name'] + '_weights.h5')
  return model
 
prediction_models = {
    'shakespeare' : load_model(basepath, shakespeare_meta),
    'casanova' : load_model(basepath, casanova_meta),
    'clarissa' : load_model(basepath, clarissa_meta),
}
prediction_models['shakespeare'].summary()

total 8
drwx------ 2 root root 4096 Mar  2 06:08 01
drwx------ 2 root root 4096 Mar  2 06:09 02
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, None, 94)          0         
_________________________________________________________________
lstm_layer_1 (LSTM)          (None, None, 640)         1881600   
_________________________________________________________________
lstm_layer_2 (LSTM)          (None, None, 640)         3279360   
_________________________________________________________________
time_distributed_4 (TimeDist (None, None, 94)          60254     
Total params: 5,221,214
Trainable params: 5,221,214
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_output(model, training_text, model_meta, start_index=None, diversity=None, amount=400):
    CHUNK_SIZE = model_meta['chunk_size']
    if start_index is None:
        start_index = random.randint(0, len(training_text) - CHUNK_SIZE - 1)
    print("start_index : %s" % start_index)
    generated = training_text[start_index: start_index + CHUNK_SIZE]
    yield generated + '#'
    for i in range(amount):
        x = np.zeros((1, len(generated), len(model_meta['char'])))
        for t, char in enumerate(generated):
            x[0, t, model_meta['char_to_idx'][char]] = 1.
        preds = model.predict(x, verbose=0)[0]
        if diversity is None:
            next_index = np.argmax(preds[len(generated) - 1])
        else:
            preds = np.asarray(preds[len(generated) - 1]).astype('float64')
            preds = np.log(preds) / diversity
            exp_preds = np.exp(preds)
            preds = exp_preds / np.sum(exp_preds)
            probas = np.random.multinomial(1, preds, 1)
            next_index = np.argmax(probas)     
        next_char = model_meta['char'][next_index]
        yield next_char

        generated += next_char
    return generated
        

카사노바 모델에 말을 시켜 보았다.

diversity = None 으로 하였더니 뭔가 서두가 비슷한 말을 자꾸 한다.

회고록으로 학습 시켜서 그런지 "대화" 문체가 많다.

사실 소설문체로 endline 없이 결과가 쭉 나올 경우가 더 많은데, 읽기 힘들어서 이걸로 골랐다.



In [0]:
for ch in generate_output(prediction_models['casanova'], casanova, model_meta=casanova_meta, amount=1000, start_index=6093314):
    sys.stdout.write(ch)
print()

start_index : 6093314
o to tell him that I wanted to start directly after dinner. This order acted on Betty like magic.

“You mean to go as far as Centino, I suppose,” said the man. #“What do you mean?”

“No, I should like to be in the same place.”

“That will do nicely.”

“I will do so, and I am sure you will be able to get a hundred thousand francs in company.”

“I will do so, and I shall not let her come and see me.”

“I will tell you the whole story.”

“You are right, and I am sure you will be able to get a hundred thousand francs in company.”

“I am delighted to hear it; the only condition is what I can do to consent to any other place. I told her that I should not have the honour of calling on you to-morrow, and I shall be delighted to see you.”

“I am glad to hear it, but I will tell you where you are going.”

“I am delighted to hear it; the only condition is what she will say when she sees me the whole affair.”

“I am glad to hear it; but you must agree with me that I am not a 

세익스피어 모델에, 상기 카사노바의 시작문장을 똑같이 넣어서 말을 시켜 보았다.

돈페드로와 클라우디오가 자꾸 Why 들어간 말을 한다(...)

희곡 처럼 구성된 문장으로 나온다.


In [0]:
for ch in generate_output(prediction_models['shakespeare'], casanova, model_meta=shakespeare_meta, amount=1000, start_index=6093314):
    sys.stdout.write(ch)
print()

start_index : 6093314
o to tell him that I wanted to start directly after dinner. This order acted on Betty like magic.

“You mean to go as far as Centino, I suppose,” said the man. #I will tell you the
sleeping of the country, and the seventh and fire; and there is a
      better place than a star, and there is more to the worst of all
      than the wind.


      DON PEDRO.
      Why, what a candle may be hang’d!


      DON PEDRO.
      Why, what a woman would that be the devil was set at the last,
      and there is no time to be a sin-wounded beard, the contrary was here at the basest.


      DON PEDRO.
      Why, what a corrust house is this! What are you?


      CLAUDIO.
      I will not hear you speak to me, I was conceived against
      you were better than your wisdom. I have been set in your way, and
      he shall not live.


      DON PEDRO.
      Why, what is he?


      DON PEDRO.
      Why, what a corrust name!


      BORACHIO.
      I cannot tell what you will not c